<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Einstein Toolkit Thorn: `NRPyEllipticET`

## Author: Leo Werneck
### Formatting improvements courtesy Brandon Clark

## This module generate black-hole puncture initial data in *SinhSymTP coordinates* (though other coordinates, including Cartesian, may be chosen).

**Notebook Status:** <font color ="red"><b> In progress </b></font>

**Validation Notes:** This module is under development.

### NRPy+ Source Code for this module: 
* [NRPyElliptic/NRPyElliptic_RHSs.py](../edit/NRPyElliptic/NRPyElliptic_RHSs.py) [\[**tutorial**\]](Tutorial-NRPyElliptic_BasicEquations.ipynb) Generates the right-hand side for the hyperbolized Hamiltonian constraint equation
* [NRPyElliptic/NRPyElliptic_SourceTerm.py](../edit/NRPyElliptic/NRPyElliptic_SourceTerm.py) [\[**tutorial**\]](Tutorial-NRPyElliptic_BasicEquations.ipynb) Generating C code for $\psi_{\rm{background}}$ and $\tilde{A}_{ij} \tilde{A}^{ij}$

## Introduction:
As outlined in the [previous NRPy+ tutorial notebook](Tutorial-NRPyElliptic_BasicEquations.ipynb), we use NRPy+ to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data.</font>
1. Evolve the system forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate RHS expressions.</font>
    1. Apply boundary conditions.
1. At the end of each iteration in time, output the relative error between numerical and exact solutions.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 0](#physical_parameters): Input physical parameters 
1. [Step 1](#writec): Generate C code to solve the hyperbolized equation in curvilinear coordinates
    1. [Step 1.a](#id_rhss): C code generation: Initial guess and scalar wave right-hand-sides
    1. [Step 1.b](#boundaryconditions): C code generation: Boundary condition driver
    1. [Step 1.c](#cparams_rfm_and_domainsize): Generate Cparameters files; set reference metric parameters, including `domain_size`
    1. [Step 1.d](#cfl): C code generation: Finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep
    1. [Step 1.e](#diagnostics): Step 1.e: Output needed C code for diagnostics 
1. [Step 2](#mainc): The C code `main()` function for `NRPyElliptic_Playground`
1. [Step 3](#compileexec): Compile generated C codes & solve the scalar wave equation
1. [Step 4](#relaxation): Relaxation: Visualizing the solution and residual in time
    1. [Step 4.a](#relaxation_residual_L2_norm): Relaxation: $L2$-norm of residual
    1. [Step 4.b](#relaxation_residual_2D): Relaxation: 2D visualization of `residual`
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='writec'></a>

# Step 1: Using NRPy+ to generate necessary C code to solve the hyperbolized equation in curvilinear, singular coordinates \[Back to [top](#toc)\]
$$\label{writec}$$

<a id='id_rhss'></a>

## Step 1.a: C code generation: Initial guess and scalar wave RHSs \[Back to [top](#toc)\]
$$\label{id_rhss}$$

We perform the relaxation with vanishing initial data, i.e.,

$$
    u(\vec{x}, 0) = v(\vec{x}, 0) = 0 \,.
$$

The RHSs of the Hamiltonian constraint equation in curvilinear coordinates (documented [in the previous module](Tutorial-NRPyElliptic_BasicEquations.ipynb)) are given by

\begin{align}
\partial_t u &= v - \eta u \\
\partial_t v &= c^2\left({\nabla}^2 u + \frac{1}{8} \tilde{A}_{ij} \tilde{A}^{ij} (\psi_{\rm{BG}} +  u)^{-7}\right),
\end{align}

where 

$$
    \psi_{\rm{singular}} = 1 + \sum_{i=1}^{N} \frac{m_{i}}{2| \vec{r} - \vec{r}_{i} |} \,,
$$

and $\tilde{A}_{ij}$ is the conformal tracefree extrinsic curvature. 

Note: In the NRPyElliptic code, the singular piece of the conformal factor, $\psi_{\rm{singular}}$, is referred to as `psi_background`.

Below we generate 
+ the RHS expressions by calling `PunctureInitialDataCurvilinear_RHSs()` inside the NRPy+ [PunctureInitialDataCurvilinear_RHSs.py](../edit//PunctureInitialData/PunctureInitialDataCurvilinear_RHSs.py) module (documented in [this NRPy+ Jupyter notebook](Tutorial-NRPyElliptic_BasicEquations.ipynb)).

In [ ]:
# Step P1: Import needed NRPy+ core modules:
import sympy as sp                    # SymPy: The Python computer algebra package upon which NRPy+ depends 
from outputC import lhrh, add_to_Cfunction_dict  # NRPy+: Core C code output module
import finite_difference as fin       # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par        # NRPy+: Parameter interface
import indexedexp as ixp              # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import grid as gri                    # NRPy+: Functions having to do with numerical grids
from outputC import outputC,lhrh,outCfunction # NRPy+: Core C code output module
import loop as lp                     # NRPy+: C code loop interface
import reference_metric as rfm        # NRPy+: Reference metric support
import cmdline_helper as cmd          # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys                # Standard Python modules for multiplatform OS-level functions

# Step P2: Create C code output directory:
thorndir = "NRPyEllipticET_autogenerated"
srcdir   = os.path.join(thorndir,"src")
cfbbhdir = os.path.join(srcdir,"conformally_flat_BBH")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
shutil.rmtree(thorndir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(thorndir)
cmd.mkdir(srcdir)
cmd.mkdir(cfbbhdir)

# Step P4: Enable/disable SIMD. If enabled, code should run ~2x faster on most CPUs.
enable_SIMD = True

# Step P5: Enable reference metric precomputation.
enable_rfm_precompute = True

if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")
else:
    par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "False")

if enable_SIMD and not enable_rfm_precompute:
    print("ERROR: SIMD does not currently handle transcendental functions,\n")
    print("       like those found in rfmstruct (rfm_precompute).\n")
    print("       Therefore, enable_SIMD==True and enable_rfm_precompute==False\n")
    print("       is not supported.\n")
    sys.exit(1)

# Step P6: Enable "FD functions". In other words, all finite-difference stencils
#         will be output as inlined static functions. This is essential for
#         compiling highly complex FD kernels with using certain versions of GCC;
#         GCC 10-ish will choke on BSSN FD kernels at high FD order, sometimes
#         taking *hours* to compile. Unaffected GCC versions compile these kernels
#         in seconds. FD functions do not slow the code performance, but do add
#         another header file to the C source tree.
# With gcc 7.5.0, enable_FD_functions=True decreases performance by 10%
enable_FD_functions = False

# Step 1: Set some core parameters, including CoordSystem, boundary condition,
#                                             MoL, timestepping algorithm, FD order,
#                                             floating point precision, and CFL factor:

# Step 1.a: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "SinhSymTP"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Step 1.b: Set boundary conditions
# Currently Sommerfeld is the only supported BC driver
OuterBoundaryCondition = "Sommerfeld"

# Step 1.c: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size     = 1.0e6 # Needed for all coordinate systems.

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.07 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 5 # If SymTP chosen

# Step 1.d: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 10        # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 0.7

In [ ]:
# Step 2: Import NRPyElliptic_RHSs module.
#         This command only declares ScalarWave RHS parameters
#         and the ScalarWave_RHSs function (called later)
import NRPyElliptic_codegen.NRPyElliptic_RHSs as puncrhs

In [ ]:
# Step 3: Import source term generation module
import NRPyElliptic_codegen.NRPyElliptic_SourceTerm as puncsource

In [ ]:
# Step 4: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

In [ ]:
# Step 5: Generate SymPy symbolic expressions for
#         uu_rhs and vv_rhs; the PunctureID RHSs.
#         This function also declares the uu and vv
#         gridfunctions, which need to be declared
#         to output even the initial data to C file.
# First get into the enable_rfm_precompute environment, if enable_rfm_precompute==True
if enable_rfm_precompute:
    cmd.mkdir(os.path.join(cfbbhdir, "rfm_files/"))
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
    par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", os.path.join(cfbbhdir, "rfm_files/"))

# Compute RHSs
puncrhs.PunctureInitialDataCurvilinear_RHSs()
# Compute alpha and beta terms that go into the source term of the Hamiltonian constraint equation
puncsource.PunctureInitialDataCurvilinear_psi_background_and_ADD_AUU_term()

# Step 5.a: Now that we are finished with all the rfm hatted
#           quantities, let's restore them to their closed-
#           form expressions.
if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()

In [ ]:
# Step 6: Copy SIMD/SIMD_intrinsics.h to $cfbbhdir/SIMD/SIMD_intrinsics.h
if enable_SIMD:
    cmd.mkdir(os.path.join(cfbbhdir,"SIMD"))
    shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(cfbbhdir,"SIMD/"))

# Step 7: Set enable "FD functions" parameter. See above for details.
par.set_parval_from_str("finite_difference::enable_FD_functions", enable_FD_functions)

# Step 8: If enable_SIMD, then copy SIMD/SIMD_intrinsics.h to $cfbbhdir/SIMD/SIMD_intrinsics.h
if enable_SIMD:
    shutil.copy(os.path.join("SIMD", "SIMD_intrinsics.h"), os.path.join(cfbbhdir, "SIMD"))

<a id='mol'></a>

## Step 1.b: Generate Method of Lines timestepping code \[Back to [top](#toc)\]
$$\label{mol}$$

The Method of Lines algorithm is described in detail in the [**NRPy+ tutorial notebook on Method of Lines algorithm**](Tutorial-Method_of_Lines-C_Code_Generation.ipynb).

In [ ]:
# Step 9: Generate Runge-Kutta-based (RK-based) timestepping code.
#       As described above the Table of Contents, this is a 2-step process:
#       9.b.A: Evaluate RHSs (RHS_string)
#       9.b.B: Apply boundary conditions (post_RHS_string, pt 1)
import MoLtimestepping.MoL_new_way as MoL

RHS_string = "rhs_eval(params, rfmstruct, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);"
if not enable_rfm_precompute:
    RHS_string = RHS_string.replace("rfmstruct", "xx")
if OuterBoundaryCondition == "Sommerfeld":
    # Sommerfeld BCs are applied to the gridfunction RHSs directly
    RHS_string += "apply_bcs_sommerfeld(params, griddata->xx, bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);"
    post_RHS_string = ""
else:
    print("Invalid choice of boundary condition")
    sys.exit(1)

MoL.register_C_functions_and_NRPy_basic_defines(RK_method,
        RHS_string=RHS_string, post_RHS_string=post_RHS_string,
        enable_rfm=enable_rfm_precompute, enable_curviBCs=True, enable_SIMD=enable_SIMD, enable_griddata=True)

In [ ]:
# Declare function for generating initial guess

def add_to_Cfunction_dict_initial_guess_single_point():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    desc = "Initial guess at a single point."
    c_type = "void"
    name = "initial_guess_single_point"
    params = """const paramstruct *restrict params,
                const REAL xx0, const REAL xx1, const REAL xx2,
                REAL *uu_exact, REAL *vv_exact"""
    body = fin.FD_outputC("returnstring",[lhrh(lhs="*uu_exact",rhs=sp.sympify(0)),
                                          lhrh(lhs="*vv_exact",rhs=sp.sympify(0))],
                          params="includebraces=False,preindent=1,outCverbose=False")
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."))

def add_to_Cfunction_dict_initial_guess_all_points():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    desc = "Initial guess at all points."
    c_type = "void"
    name = "initial_guess_all_points"
    params = "const paramstruct *restrict params, REAL *restrict xx[3], REAL *restrict in_gfs"
    body = """initial_guess_single_point(params, xx0, xx1, xx2,
           &in_gfs[IDX4S(UUGF,i0,i1,i2)], &in_gfs[IDX4S(VVGF,i0,i1,i2)]);"""
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = "AllPoints,Read_xxs")

In [ ]:
# Declare function for source terms
def add_to_Cfunction_dict_auxevol_gfs_all_points():
    desc="Set auxevol_gfs at all points"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    if enable_FD_functions:
        includes += ["finite_difference_functions.h"]
    c_type = "void"
    name = "auxevol_gfs_all_points"
    params   ="const paramstruct *restrict params, REAL *restrict xx[3], REAL *restrict auxevol_gfs"
    body = fin.FD_outputC("returnstring",
                          [lhrh(lhs=gri.gfaccess("auxevol_gfs","psi_background"),rhs=puncsource.psi_background),
                           lhrh(lhs=gri.gfaccess("auxevol_gfs","ADD_times_AUU"), rhs=puncsource.ADD_times_AUU)],
                          params="enable_SIMD=False")
    loopopts = "AllPoints,Read_xxs"
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = loopopts)

In [ ]:
# Declare function for setting up spatially dependent wavespeed
def add_to_Cfunction_dict_wavespeed_gf_all_points():

    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]

    # Compute proper distance in all 3 directions.
    delxx = ixp.declarerank1("dxx", DIM=3)
    ds_drn = rfm.ds_dirn(delxx)
    params_str = "includebraces=False"
    ds_dirn_h = outputC([ds_drn[0], ds_drn[1], ds_drn[2]], ["ds_dirn0", "ds_dirn1", "ds_dirn2"],
                        "returnstring", params=params_str)

    desc="Maximum CFL-constrained wavespeed at all points"


    add_to_Cfunction_dict(
        includes=includes,
        desc     =desc,
        c_type   ="void",
        name     ="wavespeed_gf_all_points",
        params   ="const paramstruct *restrict params, const REAL CFL_FACTOR, const REAL dt, REAL *restrict xx[3], REAL *restrict auxevol_gfs",
        preloop  ="",
        body     ="REAL ds_dirn0, ds_dirn1, ds_dirn2;\n"+ds_dirn_h+"""
#ifndef MIN
#define MIN(A, B) ( ((A) < (B)) ? (A) : (B) )
#endif
// Set dsmin = minimum proper distance;
REAL dsmin = MIN(ds_dirn0,MIN(ds_dirn1,ds_dirn2));
"""
+" "*0 + gri.gfaccess("auxevol_gfs","wavespeed") + " = CFL_FACTOR*dsmin/dt;",
        rel_path_to_Cparams=os.path.join("."),
#         loopopts ="InteriorPoints,Read_xxs,DisableOpenMP",
        loopopts="InteriorPoints,Read_xxs",
        postloop ="")

In [ ]:
# Declare function for RHSs evaluation

def add_to_Cfunction_dict_rhs_eval():
    desc="Evaluate the RHSs of the Hamiltonian constraint equation"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    if enable_FD_functions:
        includes += ["finite_difference_functions.h"]
    if enable_SIMD:
        includes += ["SIMD/SIMD_intrinsics.h"]
    c_type = "void"
    name = "rhs_eval"
    params   ="const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct *restrict rfmstruct, "
    else:
        params += "REAL *restrict xx[3], "
    params += "const REAL *restrict auxevol_gfs, const REAL *restrict in_gfs, REAL *restrict rhs_gfs"
    body = fin.FD_outputC("returnstring",[lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=puncrhs.uu_rhs),
                                          lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=puncrhs.vv_rhs)],
                          params="enable_SIMD="+str(enable_SIMD))
    loopopts = "InteriorPoints"
    if enable_SIMD:
        loopopts += ",enable_SIMD"
    if enable_rfm_precompute:
        loopopts += ",enable_rfm_precompute"
    else:
        loopopts += ",Read_xxs"
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = loopopts)

<a id='boundaryconditions'></a>

## Step 1.b: Output needed C code for boundary condition driver \[Back to [top](#toc)\]
$$\label{boundaryconditions}$$

In [ ]:
import CurviBoundaryConditions.CurviBoundaryConditions_new_way as CBC
CBC.CurviBoundaryConditions_register_C_functions_and_NRPy_basic_defines()
CBC.add_to_Cfunction_dict_set_up__bc_gz_map_and_parity_condns()

In [ ]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
if OuterBoundaryCondition == "Sommerfeld":
    bcs = cbcs.sommerfeld_boundary_condition_class(fd_order=6,
                                                 vars_radial_falloff_power_default=3,
                                                 vars_speed_default=1.0,
                                                 vars_at_inf_default=0.)
    bcs.write_sommerfeld_file(cfbbhdir)

In [ ]:
# Declare function to update wavespeed at outer boundary

def add_to_Cfunction_dict_update_evolgf_speed():
    desc="Update evolgf_speed at outer boundary"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    name = "update_evolgf_speed"
    params  = "const paramstruct *restrict params, REAL *restrict auxevol_gfs, REAL *evolgf_speed" 
    body = r"""
  REAL wavespeed_at_OB = auxevol_gfs[IDX4S(WAVESPEEDGF, Nxx_plus_2NGHOSTS0 - NGHOSTS - 1, 
                                     NGHOSTS, Nxx_plus_2NGHOSTS2/2)];
  for (int ii = 0; ii < NUM_EVOL_GFS; ii++){
    evolgf_speed[ii] = wavespeed_at_OB;
  } // END for
"""     
         
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params, body=body,
        rel_path_to_Cparams=os.path.join("."))

<a id='cparams_rfm_and_domainsize'></a>

## Step 1.c: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams_rfm_and_domainsize}$$

Here we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [ ]:
# Step 1.c.i: Set free_parameters.h
outstr = r"""
// Free parameters related to physical system:
params.time = 0.0; // Initial simulation time corresponds to time=0.

// Free parameters related to numerical timestep:
REAL CFL_FACTOR = """+str(CFL_FACTOR)+";\n"

# Append to $cfbbhdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
outstr += rfm.out_default_free_parameters_for_rfm("returnstring",
                                                  domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

# Append to $cfbbhdir/free_parameters.h puncture-related parameters
# outstr += r"""
# // Free parameters related to punctures
# """
# for param, value in params_dict.items():
#     outstr += f"params.{param} = {value};\n"

# with open(os.path.join(cfbbhdir,"free_parameters.h"),"w") as file:
#     file.write(outstr.replace("params.", "griddata.params."))

<a id='cfl'></a>

## Step 1.d: Output needed C code for finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep \[Back to [top](#toc)\]
$$\label{cfl}$$

In order for our explicit-timestepping numerical solution to the scalar wave equation to be stable, it must satisfy the [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673) condition:
$$
\Delta t \le \frac{\min(ds_i)}{c},
$$
where $c$ is the wavespeed, and
$$ds_i = h_i \Delta x^i$$ 
is the proper distance between neighboring gridpoints in the $i$th direction (in 3D, there are 3 directions), $h_i$ is the $i$th reference metric scale factor, and $\Delta x^i$ is the uniform grid spacing in the $i$th direction:

In [ ]:
# Generate & register C function set_Nxx_dxx_invdx_params__and__xx()
# Generate & register C function xx_to_Cart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
# Generate & register the find_timestep() function
rfm.register_C_functions(enable_rfm_precompute=enable_rfm_precompute, use_unit_wavespeed_for_find_timestep=True)
rfm.register_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)

<a id='diagnostics'></a>

## Step 1.e: Output needed C code for diagnostics \[Back to [top](#toc)\]
$$\label{diagnostics}$$

In [ ]:
# Declare function for residual evaluation

def add_to_Cfunction_dict_residual_all_points():
    desc="Evaluate the residual at all points"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    if enable_FD_functions:
        includes += ["finite_difference_functions.h"]
    if enable_SIMD:
        includes += ["SIMD/SIMD_intrinsics.h"]
    c_type = "void"
    name = "residual_all_points"
    params   ="const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct *restrict rfmstruct, "
    else:
        params += "REAL *restrict *xx, "
    params += "const REAL *restrict auxevol_gfs, const REAL *restrict in_gfs, REAL *restrict residual_gf"
    body = fin.FD_outputC("returnstring",[lhrh(lhs=gri.gfaccess("residual_gf","uu"),rhs=puncrhs.residual)],
                          params="enable_SIMD="+str(enable_SIMD))
    loopopts = "InteriorPoints"
    if enable_SIMD:
        loopopts += ",enable_SIMD"
    if enable_rfm_precompute:
        loopopts += ",enable_rfm_precompute"
    else:
        loopopts += ",Read_xxs"
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = loopopts)

In [ ]:
# Declare function for L2-norm computation

def add_to_Cfunction_dict_L2_norm_of_gf():
    desc="Evaluate L2-norm of gridfunction"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "REAL"
    name = "L2_norm_of_gf"
    params  = "const paramstruct *restrict params, const int gf_index, const REAL integration_radius, " 
    params += "REAL *restrict xx[3], const REAL *restrict in_gf"
    preloop = r"""
  REAL squared_sum = 0.0;
  REAL volume_sum  = 0.0;
    """
    body = outputC([rfm.xxSph[0], sp.sqrt(rfm.detgammahat)], ["const REAL r", "const REAL sqrtdetgamma"],
                   filename="returnstring", params="includebraces=False,outCverbose=False,preindent=0")
    body += r"""
if(r < integration_radius) {
  const REAL gf_of_x = in_gf[IDX4S(gf_index,i0,i1,i2)];
  const REAL dV = sqrtdetgamma*dxx0*dxx1*dxx2;
  squared_sum += gf_of_x*gf_of_x*dV;
  volume_sum  += dV;
} // END if(r < integration_radius)
"""     
    loopopts = "InteriorPoints,Read_xxs,OMP_custom_pragma='#pragma omp parallel for reduction(+:squared_sum,volume_sum)'"
    
    postloop = r"""
  // Compute and output the log of the L2 norm.
  return log10(1e-16 + sqrt(squared_sum/volume_sum));  // 1e-16 + ... avoids log10(0)
"""
         
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        preloop=preloop, body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = loopopts, postloop=postloop)

In [ ]:
# Declare function for outputting gridfunction along the z-axis assuming *SymTP coordinates

def add_to_Cfunction_dict_gridfunction_z_axis():
    desc="Output gridfunction along z axis"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    name = "gridfunction_z_axis"
    params  = "const paramstruct *restrict params, const int gf_index, REAL *restrict xx[3], "
    params += "const REAL *restrict in_gf, FILE *restrict outfile"
    
    output_on_z_axis_str  = outputC([rfm.xx_to_Cart[2]], ["const REAL z"],
                            filename="returnstring", params="includebraces=False,outCverbose=False,preindent=2")
    output_on_z_axis_str += """
    const REAL gf_at_z = in_gf[IDX4S(gf_index, i0, i1, i2)];
    fprintf(outfile, "%.17e %.17e\\n", z, gf_at_z);
"""
    
    body = """
  // Set i2 to any index (z-axis is independent of i2)
  const int i2 = Nxx_plus_2NGHOSTS2/2; const REAL xx2 = xx[2][i2];
  
  // Output postive z-axis
  // Set x1 = 0 (i1 = NGHOSTS)
  int i1 = NGHOSTS; REAL xx1 = xx[1][i1];
  for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++){
    REAL xx0 = xx[0][i0];
""" + output_on_z_axis_str
    body += """
  } // END LOOP: for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++)
"""
    
    body += """
  // Output negative z-axis
  // Set x1 = pi (i1 = Nxx_plus_2NGHOSTS1 - NGHOSTS)
  i1 = Nxx_plus_2NGHOSTS1 - NGHOSTS - 1; xx1 = xx[1][i1];
  for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++){
    REAL xx0 = xx[0][i0];
""" + output_on_z_axis_str
    body += """
  } // END LOOP: for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++)
"""
    
    body += """
  // Add i0 = NGHOSTS line (x0 = 0)
  // Here, we do not add the points i1 = NGHOSTS and i1 = xx_plus_2NGHOSTS1 - NGHOSTS,
  // as they hare already been added in the previous loops
  int i0 = NGHOSTS; REAL xx0 = xx[0][i0];
  for (int i1 = NGHOSTS + 1; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS - 1; i1++){
    REAL xx1 = xx[1][i1];
""" + output_on_z_axis_str
    body += """
  } // END LOOP: for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++)
"""

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        preloop="", body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = "", postloop="")

In [ ]:
# Declare function for outputting gridfunction along the xz-plane assuming *SymTP coordinates

def add_to_Cfunction_dict_gridfunction_xz_plane():
    desc="Output gridfunction along xz-plane"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    name = "gridfunction_xz_plane"
    params  = "const paramstruct *restrict params, const int gf_index, REAL *restrict xx[3], "
    params += "const REAL *restrict in_gf, FILE *restrict outfile"
    
    output_on_xz_plane_str  = outputC([rfm.xx_to_Cart[0], rfm.xx_to_Cart[2]], ["const REAL xCart", "const REAL zCart"],
                            filename="returnstring", params="includebraces=False,outCverbose=False,preindent=3")
    output_on_xz_plane_str += """
      const REAL gf_at_xz = in_gf[IDX4S(gf_index, i0, i1, i2)];
      fprintf(outfile, "%.17e %.17e %.17e\\n", xCart, zCart, gf_at_xz);
"""
    
    body = """
  // Set x2 = -pi (i2 = NGHOSTS) and loop over i0 and i1
  int i2 = NGHOSTS; REAL xx2 = xx[2][i2];
  
  // Output points with x < 0
  for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++){
    REAL xx1 = xx[1][i1];
    for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++){
      REAL xx0 = xx[0][i0];
""" + output_on_xz_plane_str +"""
    } // END for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++)
  } // END for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++)
"""
    
    body += """
  // Set x2 = 0 (i2 = Nxx_plus_2NGHOSTS2/2) and loop over i0 and i1
  i2 = Nxx_plus_2NGHOSTS2/2; xx2 = xx[2][i2];
  
  // Output points with x > 0
  for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++){
    REAL xx1 = xx[1][i1];
    for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++){
      REAL xx0 = xx[0][i0];
""" + output_on_xz_plane_str + """
    } // END for (int i0 = NGHOSTS; i0 < Nxx_plus_2NGHOSTS0 - NGHOSTS; i0++)
  } // END for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++)
"""

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        preloop="", body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = "", postloop="")

In [ ]:
# Declare function for outputting parameters

def add_to_Cfunction_dict_print_puncture_parameters():
    desc="Print all puncture parameters"
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    name = "print_puncture_parameters"
    params  = "const paramstruct *restrict params"
    body = """
  printf("bScale       = %lf\\n", bScale);
  printf("SINHWAA      = %lf\\n", SINHWAA);
  printf("AMAX         = %lf\\n", AMAX);
  printf("eta_damping  = %lf\\n", eta_damping);
  printf("bare_mass_0  = %lf\\n", bare_mass_0);
  printf("bare_mass_1  = %lf\\n", bare_mass_1);
  printf("puncture_0_x = %lf\\n", puncture_0_x);
  printf("puncture_0_y = %lf\\n", puncture_0_y);
  printf("puncture_0_z = %lf\\n", puncture_0_z);
  printf("puncture_1_x = %lf\\n", puncture_1_x);
  printf("puncture_1_y = %lf\\n", puncture_1_y);
  printf("puncture_1_z = %lf\\n", puncture_1_z);
  printf("P0_x         = %lf\\n", P0_x);
  printf("P0_y         = %lf\\n", P0_y);
  printf("P0_z         = %lf\\n", P0_z);
  printf("P1_x         = %lf\\n", P1_x);
  printf("P1_y         = %lf\\n", P1_y);
  printf("P1_z         = %lf\\n", P1_z);
  printf("S0_x         = %lf\\n", S0_x);
  printf("S0_y         = %lf\\n", S0_y);
  printf("S0_z         = %lf\\n", S0_z);
  printf("S1_x         = %lf\\n", S1_x);
  printf("S1_y         = %lf\\n", S1_y);
  printf("S1_z         = %lf\\n", S1_z);
  printf("Nxx0         = %d\\n",  Nxx0);
  printf("Nxx1         = %d\\n",  Nxx1);
  printf("Nxx2         = %d\\n",  Nxx2);
  printf("NGHOSTS      = %d\\n",  NGHOSTS);
"""
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        preloop="", body=body,
        rel_path_to_Cparams=os.path.join("."), loopopts = "", postloop="")

<a id='mainc'></a>

# Step 2: The hyperbolic relaxation function for conformally flat BBH initial data \[Back to [top](#toc)\]
$$\label{mainc}$$

In [ ]:
def add_to_Cfunction_dict_NRPyEllipticET_conformally_flat_BBH_Hyperbolic_Relaxation():
    includes = ["../NRPyEllipticET.h", "NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    desc   = ""
    c_type = "void"
    name   = "NRPyEllipticET_conformally_flat_BBH_Hyperbolic_Relaxation"
    params = ""
    body   = r"""
  DECLARE_CCTK_PARAMETERS;

  // Set the size of the local grid
  const CCTK_INT Nxx_plus_2NGHOSTS0    = N0 + 2*NGHOSTS;
  const CCTK_INT Nxx_plus_2NGHOSTS1    = N1 + 2*NGHOSTS;
  const CCTK_INT Nxx_plus_2NGHOSTS2    = N2 + 2*NGHOSTS;
  const CCTK_INT Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

  // Allocate memory for NRPyEllipticET_xx and NRPyEllipticET_uu
  NRPyEllipticET_xx[0] = (CCTK_REAL *)malloc(sizeof(CCTK_REAL)*Nxx_plus_2NGHOSTS0);
  NRPyEllipticET_xx[1] = (CCTK_REAL *)malloc(sizeof(CCTK_REAL)*Nxx_plus_2NGHOSTS1);
  NRPyEllipticET_xx[2] = (CCTK_REAL *)malloc(sizeof(CCTK_REAL)*Nxx_plus_2NGHOSTS2);
  NRPyEllipticET_uu    = (CCTK_REAL *)malloc(sizeof(CCTK_REAL)*Nxx_plus_2NGHOSTS_tot);

  // Check everything is okay
  if( !NRPyEllipticET_uu || !NRPyEllipticET_xx[0] || !NRPyEllipticET_xx[1] || !NRPyEllipticET_xx[2] ) {
    CCTK_ERROR("Could not allocate memory for NRPyEllipticET_uu or NRPyEllipticET_xx!");
  }

  // Step 0.a: Check grid parameters
  const int Nxx[3] = { N0,N1,N2 };
  if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
    CCTK_ERROR("Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number."
               " For example, in case of angular directions, proper symmetry zones will not exist.");
  }

  // Step 0.b: Print parameter information
  CCTK_INFO("Information about puncture 1:");
  CCTK_VINFO("    Bare mass        = %10.3e", conformally_flat_BBH_puncture_0_bare_mass);
  CCTK_VINFO("    Position x       = %10.3e", conformally_flat_BBH_puncture_0_pos[0]);
  CCTK_VINFO("    Position y       = %10.3e", conformally_flat_BBH_puncture_0_pos[1]);
  CCTK_VINFO("    Position z       = %10.3e", conformally_flat_BBH_puncture_0_pos[2]);
  CCTK_VINFO("    Lin. momentum x  = %10.3e", conformally_flat_BBH_puncture_0_P[0]);
  CCTK_VINFO("    Lin. momentum y  = %10.3e", conformally_flat_BBH_puncture_0_P[1]);
  CCTK_VINFO("    Lin. momentum z  = %10.3e", conformally_flat_BBH_puncture_0_P[2]);
  CCTK_VINFO("    Ang. momentum x  = %10.3e", conformally_flat_BBH_puncture_0_S[0]);
  CCTK_VINFO("    Ang. momentum y  = %10.3e", conformally_flat_BBH_puncture_0_S[1]);
  CCTK_VINFO("    Ang. momentum z  = %10.3e", conformally_flat_BBH_puncture_0_S[2]);

  CCTK_INFO("Information about puncture 2:");
  CCTK_VINFO("    Bare mass        = %10.3e", conformally_flat_BBH_puncture_1_bare_mass);
  CCTK_VINFO("    Position x       = %10.3e", conformally_flat_BBH_puncture_1_pos[0]);
  CCTK_VINFO("    Position y       = %10.3e", conformally_flat_BBH_puncture_1_pos[1]);
  CCTK_VINFO("    Position z       = %10.3e", conformally_flat_BBH_puncture_1_pos[2]);
  CCTK_VINFO("    Lin. momentum x  = %10.3e", conformally_flat_BBH_puncture_1_P[0]);
  CCTK_VINFO("    Lin. momentum y  = %10.3e", conformally_flat_BBH_puncture_1_P[1]);
  CCTK_VINFO("    Lin. momentum z  = %10.3e", conformally_flat_BBH_puncture_1_P[2]);
  CCTK_VINFO("    Ang. momentum x  = %10.3e", conformally_flat_BBH_puncture_1_S[0]);
  CCTK_VINFO("    Ang. momentum y  = %10.3e", conformally_flat_BBH_puncture_1_S[1]);
  CCTK_VINFO("    Ang. momentum z  = %10.3e", conformally_flat_BBH_puncture_1_S[2]);

  // Step 0.c: Set griddata struct
  griddata_struct griddata;
  conformally_flat_BBH_set_Cparameters_to_default(&griddata.params);

  // Step 0.d: Set parameters from parfile
#include "conformally_flat_BBH_set_Cparameters_from_parfile.h"

  // Step 0.e: Update parameters if using xy-plane as the orbital plane
  if( CCTK_EQUALS(orbital_plane,"xy") || CCTK_EQUALS(orbital_plane,"yx") ) {
    // The mapping is (x,y,z)_{NRPy} = (y,z,x)_{ETK}.
    // Parameters for puncture #1
    griddata.params.puncture_0_x   = conformally_flat_BBH_puncture_0_pos[1];
    griddata.params.puncture_0_y   = conformally_flat_BBH_puncture_0_pos[2];
    griddata.params.puncture_0_z   = conformally_flat_BBH_puncture_0_pos[0];
    griddata.params.puncture_0_P_x = conformally_flat_BBH_puncture_0_P[1];
    griddata.params.puncture_0_P_y = conformally_flat_BBH_puncture_0_P[2];
    griddata.params.puncture_0_P_z = conformally_flat_BBH_puncture_0_P[0];
    griddata.params.puncture_0_S_x = conformally_flat_BBH_puncture_0_S[1];
    griddata.params.puncture_0_S_y = conformally_flat_BBH_puncture_0_S[2];
    griddata.params.puncture_0_S_z = conformally_flat_BBH_puncture_0_S[0];
    // Parameters for puncture #2
    griddata.params.puncture_1_x   = conformally_flat_BBH_puncture_1_pos[1];
    griddata.params.puncture_1_y   = conformally_flat_BBH_puncture_1_pos[2];
    griddata.params.puncture_1_z   = conformally_flat_BBH_puncture_1_pos[0];
    griddata.params.puncture_1_P_x = conformally_flat_BBH_puncture_1_P[1];
    griddata.params.puncture_1_P_y = conformally_flat_BBH_puncture_1_P[2];
    griddata.params.puncture_1_P_z = conformally_flat_BBH_puncture_1_P[0];
    griddata.params.puncture_1_S_x = conformally_flat_BBH_puncture_1_S[1];
    griddata.params.puncture_1_S_y = conformally_flat_BBH_puncture_1_S[2];
    griddata.params.puncture_1_S_z = conformally_flat_BBH_puncture_1_S[0];
  }

  // Step 0.f: Uniform coordinate grids are stored to *xx[3]
  // Step 0.f.i: Set bcstruct
  {
    int EigenCoord = 1;
    // Step 0.f.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //             chosen Eigen-CoordSystem.
    conformally_flat_BBH_set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);
    // Step 0.g: Find ghostzone mappings; set up bcstruct
    conformally_flat_BBH_driver_bcstruct(&griddata.params, &griddata.bcstruct, griddata.xx);
    // Step 0.g.i: Free allocated space for xx[][] array
    for(int i=0;i<3;i++) free(griddata.xx[i]);
  }

  // Step 0.h: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
  //           params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
  //           chosen (non-Eigen) CoordSystem.
  int EigenCoord = 0;
  conformally_flat_BBH_set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);

  // Step 0.i: Set timestep based on smallest proper distance between gridpoints and CFL factor
  REAL dt = conformally_flat_BBH_find_timestep(&griddata.params, griddata.xx, CFL_FACTOR);

  // Step 0.j: Set maximum number of time steps
  int N_final = max_iterations;

  // Step 0.l: Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
  //              This is a limitation of the RK method. You are always welcome to declare & allocate
  //              additional gridfunctions by hand.
  if(NUM_AUX_GFS > NUM_EVOL_GFS) {
    CCTK_ERROR("NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,"
               " or allocate (malloc) by hand storage for *diagnostic_output_gfs.");
  }

  // Step 0.m: Declare struct for gridfunctions and allocate memory for y_n_gfs gridfunctions
  conformally_flat_BBH_MoL_malloc_y_n_gfs(&griddata.params, &griddata.gridfuncs);
"""
    if enable_rfm_precompute:
        body += """
  // Step 0.m: Set up precomputed reference metric arrays
  // Step 0.m.i: Allocate space for precomputed reference metric arrays.
  conformally_flat_BBH_rfm_precompute_rfmstruct_malloc(&griddata.params, &griddata.rfmstruct);\n"""
    body += r"""
  // Step 0.n.ii: Define precomputed reference metric arrays.
  conformally_flat_BBH_rfm_precompute_rfmstruct_define(&griddata.params, griddata.xx, &griddata.rfmstruct);

  // Step 1: Set up initial guess
  conformally_flat_BBH_initial_guess_all_points(&griddata.params, griddata.xx, griddata.gridfuncs.y_n_gfs);

  // Step 2: Allocate memory for non y_n_gfs. We do this here to free up
  //         memory for setting up initial data (for cases in which initial
  //         data setup is memory intensive.)
  conformally_flat_BBH_MoL_malloc_non_y_n_gfs(&griddata.params, &griddata.gridfuncs);

  // Step 3: Set up auxevol gridfunctions (psi_background and ADD_times_AUU)
  conformally_flat_BBH_auxevol_gfs_all_points(&griddata.params, griddata.xx, griddata.gridfuncs.auxevol_gfs);

  // Step 4: Set up wavespeed gridfunction
  conformally_flat_BBH_wavespeed_gf_all_points(&griddata.params, CFL_FACTOR, dt, griddata.xx, griddata.gridfuncs.auxevol_gfs);

  // Step 5. Set eta damping
  if( eta_damping != -1 ) {
    griddata.params.eta_damping = eta_damping;
  }
  else {
    if( domain_size==1e6 && sinh_width==0.07 && foci_position==5 ) {
      conformally_flat_BBH_set_optimum_eta_damping(dt, CFL_FACTOR, &griddata.params);
    }
    else {
      CCTK_VERROR("Automatic computation of the optimum eta_damping parameter is only available if domain_size = 1e6 (got %e), sinh_width = 0.07 (got %e) and foci_position = 5 (got %e). Please set NRPyEllipticET::eta_damping manually on your parfile or adjust your grid parameters.",domain_size, sinh_width, foci_position);
    }
  }

  // Step 6: Print more useful information
  CCTK_INFO("Grid information:");
  CCTK_VINFO("    Points in xx0    =  %d",Nxx[0]);
  CCTK_VINFO("    Points in xx1    =  %d",Nxx[1]);
  CCTK_VINFO("    Points in xx2    =  %d",Nxx[2]);
  CCTK_VINFO("    Domain size      = %10.3e",domain_size);
  CCTK_VINFO("    sinh width       = %10.3e",sinh_width);

  CCTK_INFO("Hyperbolic relaxation information:");
  CCTK_VINFO("    Damping strength = %10.3e",griddata.params.eta_damping);
  CCTK_VINFO("    Target residual  = %10.3e",pow(10.0,log_target_residual));
  CCTK_VINFO("    Time step        = %10.3e",dt);
  CCTK_VINFO("    Final time       = %10.3e",number_of_LCT*domain_size);
  CCTK_VINFO("    Max iterations   =  %d"   ,N_final);

  // Step 7: Relaxation time loop
  // Step 7.a: Start timing for NRPyEllipticET
  const clock_t start = clock();

  // Step 7.b: Set frequency with which the L2-norm of residual is computed
  int output_l2_norm_residual_every_N = info_output_freq; //= (int)((REAL)N_final/1000.0);
  if( output_l2_norm_residual_every_N == 0 ) output_l2_norm_residual_every_N = 1;

  int n;
  for(n=0;n<=N_final;n++) { // Main loop to progress forward in time.

    // Step 7.c: Step forward one timestep (t -> t+dt) in time using
    //           chosen RK-like MoL timestepping algorithm
    conformally_flat_BBH_MoL_step_forward_in_time(&griddata, dt);

    // Step 7.d: Check convergence & print information
    if( (n%output_l2_norm_residual_every_N == 0) || (n==N_final) ) {

      // Step 7.d.i: Compute residual and store it at diagnostic_output_gfs(UUFG)"""
    if enable_rfm_precompute:
        body +="""
      conformally_flat_BBH_residual_all_points(&griddata.params, &griddata.rfmstruct, griddata.gridfuncs.auxevol_gfs,
                                               griddata.gridfuncs.y_n_gfs, griddata.gridfuncs.diagnostic_output_gfs);
"""
    else:
        body +="""
      conformally_flat_BBH_residual_all_points(&griddata.params, griddata.xx, griddata.gridfuncs.auxevol_gfs,
                                               griddata.gridfuncs.y_n_gfs, griddata.gridfuncs.diagnostic_output_gfs);
"""
    body+=r"""
      const int residual_gf_index   = UUGF;
      const REAL log_l2_norm_residual = conformally_flat_BBH_L2_norm_of_gf(&griddata.params, residual_gf_index, residual_integration_radius,
                                                                           griddata.xx, griddata.gridfuncs.diagnostic_output_gfs);
      if( verbose && info_output_freq > 0 ) {
        // Step 7.d.ii: Compute time elapsed during the executation of this thorn
        const clock_t end = clock();
        REAL time_elapsed_s = (end-start)/((REAL)CLOCKS_PER_SEC);

        // Step 7.d.iii: Compute percentage of the run that has completed
        REAL completion = ((REAL)n)/((REAL)N_final);

        // Step 7.d.iv: Estimate how much time is left
        REAL time_left_s = time_elapsed_s*(1.0/completion - 1.0);

        // Step 7.d.v: Print information to the user
        CCTK_VINFO("It: %05d, t: %4.2lf, completion: %5.1lf%%, runtime: %3.0lf s, ETA: %3.0lf s, log(L2 norm H): %.2lf",
                   n, griddata.params.time, completion*100, time_elapsed_s, time_left_s, log_l2_norm_residual);
      }

      if( log_l2_norm_residual < log_target_residual ) {
        CCTK_VINFO("Target log(residual) (%.2lf) has been met: %.2lf",log_target_residual, log_l2_norm_residual);
        // We're done!
        n = 10*N_final;
        break;
      }
    }
  } // End main loop to progress forward in time.
  
  if( n==N_final ) {
    // This means the stopping criterion was not met. Error out.
    CCTK_ERROR("Solution did not reach the specified tolerance in the given number of iterations. Please increase max_iterations or the convergence criterion on the parfile.");
  }

  // Step 8: Initialize NRPyEllipticET_xx
  for(int i0=0;i0<Nxx_plus_2NGHOSTS0;i0++) NRPyEllipticET_xx[0][i0] = griddata.xx[0][i0];
  for(int i1=0;i1<Nxx_plus_2NGHOSTS1;i1++) NRPyEllipticET_xx[1][i1] = griddata.xx[1][i1];
  for(int i2=0;i2<Nxx_plus_2NGHOSTS2;i2++) NRPyEllipticET_xx[2][i2] = griddata.xx[2][i2];

  // Step 9: Initialize NRPyEllipticET_uu
#pragma omp parallel for
  LOOP_REGION(0,Nxx_plus_2NGHOSTS0,
              0,Nxx_plus_2NGHOSTS1,
              0,Nxx_plus_2NGHOSTS2) {
    NRPyEllipticET_uu[IDX4S(UUGF,i0,i1,i2)] = griddata.gridfuncs.y_n_gfs[IDX4S(UUGF,i0,i1,i2)];
  }

  // Step 10: Print final information about the run
  const clock_t end = clock();
  CCTK_VINFO("Total execution time: %.0lf s", (end-start)/((REAL)CLOCKS_PER_SEC));

  // Step 11: Free all allocated memory
"""
    if enable_rfm_precompute:
        body += "  conformally_flat_BBH_rfm_precompute_rfmstruct_freemem(&griddata.params, &griddata.rfmstruct);\n"
    body += r"""
  conformally_flat_BBH_freemem_bcstruct(&griddata.params, &griddata.bcstruct);
  conformally_flat_BBH_MoL_free_memory_y_n_gfs(&griddata.params, &griddata.gridfuncs);
  conformally_flat_BBH_MoL_free_memory_non_y_n_gfs(&griddata.params, &griddata.gridfuncs);
  for(int i=0;i<3;i++) free(griddata.xx[i]);
"""
    # As rfmstruct stores functions of xx, when rfm_precompute is disabled,
    #   we always pass xx to a function instead of &rfmstruct.
    if not enable_rfm_precompute:
        body = body.replace("&rfmstruct", "xx") #FIXME I noticed a few broken things with  rfm_precompute disabled.

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)

Next, register all remaining C functions in `outC_function_dict`, and output `finite_difference_functions.h`. Also construct `NRPy_basic_defines.h`.

In [ ]:
def register_C_code_functions_NRPyElliptic():
    add_to_Cfunction_dict_initial_guess_single_point()
    add_to_Cfunction_dict_initial_guess_all_points()
    add_to_Cfunction_dict_rhs_eval()
    add_to_Cfunction_dict_auxevol_gfs_all_points()
    add_to_Cfunction_dict_wavespeed_gf_all_points()
    add_to_Cfunction_dict_residual_all_points()
    add_to_Cfunction_dict_L2_norm_of_gf()
    add_to_Cfunction_dict_gridfunction_z_axis()
    add_to_Cfunction_dict_gridfunction_xz_plane()
    add_to_Cfunction_dict_print_puncture_parameters()
    add_to_Cfunction_dict_update_evolgf_speed()
    add_to_Cfunction_dict_NRPyEllipticET_conformally_flat_BBH_Hyperbolic_Relaxation()

import outputC as outC
outC.outputC_register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
# Declare paramstruct, register set_Cparameters_to_default(),
#   and output declare_Cparameters_struct.h and set_Cparameters[].h:
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(cfbbhdir))

gri.register_C_functions_and_NRPy_basic_defines(enable_rfmstruct=enable_rfm_precompute)  # #define IDX3S(),  etc.
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=False)  # #define NGHOSTS, etc.

# all functions needed for solver:
register_C_code_functions_NRPyElliptic()

# Output functions for computing all finite-difference stencils.
#   Must be called after defining all functions depending on FD stencils.
if enable_FD_functions:
    fin.output_finite_difference_functions_h(path=cfbbhdir)

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(cfbbhdir, enable_SIMD=enable_SIMD)
outC.construct_NRPy_function_prototypes_h(cfbbhdir)

In [ ]:
with open(os.path.join(cfbbhdir,"NRPy_basic_defines.h"),"a") as file:
    file.write("static REAL evolgf_speed[NUM_EVOL_GFS];")

<a id='compileexec'></a>

# Step 3: Generated C code \[Back to [top](#toc)\]
$$\label{compileexec}$$

To aid in the cross-platform-compatible (with Windows, MacOS, & Linux) compilation and execution, we make use of `cmdline_helper` [(**Tutorial**)](Tutorial-cmdline_helper.ipynb).

In [ ]:
from outputC import outC_function_dict, indent_Ccode

prefix = cfbbhdir.split("/")[-1]+"_"

# Function name replacements
replacements = [("apply_bcs_sommerfeld",prefix+"apply_bcs_sommerfeld")]
for name in outC_function_dict.keys():
    if "conformally_flat_BBH" not in name:
        replacements.append((name, prefix+name))

def replace_all(replacements, string):
    """ Applies multiple replacements to a string.

        Arguments
        ---------
        replacements : list of tuples
            Contains all replacements in the format [("old1","new1"), ("old2","new2"), ...].
        string : str
            String to which all replacements are applied.

        Returns
        -------
        newstring : str
            Input string with all replacements applied.
    """
    
    newstring = string
    for r in replacements:
        newstring = newstring.replace(*r)
    return newstring

def replace_all_in_file(filepath, replacements):
    """ Applies multiple replacements to the contents of a file.

        Arguments
        ---------
        filepath : str
            Path to the file.
        replacements : list of tuples
            Contains all replacements in the format [("old1","new1"), ("old2","new2"), ...].
    """

    with open(filepath, "r+") as file:
        string = file.read()
        file.seek(0)
        file.write(replace_all(replacements, string))

def replace_all_and_rename_file(oldpath, newpath, replacements):
    """ Applies multiple replacements to the contents of a file and renames the file.

        Arguments
        ---------
        oldpath : str
            Path to the old file.
        newpath : str
            Path to the new file.
        replacements : list of tuples
            Contains all replacements in the format [("old1","new1"), ("old2","new2"), ...].
    """

    replace_all_in_file(oldpath, replacements)
    os.rename(oldpath, newpath)
    

def replace_all_and_output_to_file(filepath, replacements, string):
    """ Apply a series of replacements to a string and write it to a file.
    
        Arguments
        ---------
        filepath : str
            Path to the output file.
        string : str
            String with the file contents, to which replacements will be applied.
        replacements : list of tuples
            Contains all replacements in the format [("old1","new1"), ("old2","new2"), ...].
    """

    with open(filepath, "w") as file:
        file.write(replace_all(replacements, string))

# Generate C code for all functions in the
# dictionary; also generate make.code.defn.
with open(os.path.join(cfbbhdir, "make.code.defn"), "w") as file:
    file.write("# Main make.code.defn file for thorn " + thorndir + "\n\n")
    file.write("# Source files in this directory that need compiled:\n")
    file.write("SRCS = \\\n")
    outstr = ""
    for name, func in outC_function_dict.items():
        if "conformally_flat_BBH" not in name:
            newname = prefix+name+".c"
            replace_all_and_output_to_file(os.path.join(cfbbhdir, newname), replacements, func)
        else:
            newname = name+".c"
            with open(os.path.join(cfbbhdir, newname), "w") as file2:
                file2.write(func)

        outstr += newname + " \\\n"
    outstr = outstr[:-2] + "\n"
    file.write(indent_Ccode(outstr, "       "))

# We'll keep this general in case we need to add more files later
filenames = [os.path.join(cfbbhdir, "NRPy_function_prototypes.h")]
for filename in filenames:
    replace_all_in_file(filename, replacements)

# We'll keep this general in case we need to add more files later
filenames = [os.path.join(cfbbhdir,"apply_bcs_sommerfeld.h")]
for i, oldpath in enumerate(filenames):
    filename = oldpath.split("/")[-1]
    newpath  = oldpath.replace(filename, prefix+filename)
    filenames[i] = (oldpath, newpath)
for oldpath, newpath in filenames:
    replace_all_and_rename_file(oldpath, newpath, replacements)

# Now replace all parameter names
plist = []
pdict = {}

def add_to_list_and_dict(plist, pdict, oldname, newname, prefix):
    plist.append((oldname, newname))
    pdict[newname] = prefix+newname

for i in range(2):
    puncture_id            = "puncture_"+str(i)+"_"
    add_to_list_and_dict(plist, pdict, "bare_mass_"+str(i), puncture_id+"bare_mass", prefix)
    for j in range(3):
        dirn = '_'+chr(ord('x')+j)
        add_to_list_and_dict(plist, pdict, "P"+str(i)+dirn, puncture_id+"P"+dirn, prefix)
        add_to_list_and_dict(plist, pdict, "S"+str(i)+dirn, puncture_id+"S"+dirn, prefix)

import glob
for filename in glob.glob(cfbbhdir+"/*.*")+glob.glob(cfbbhdir+"/*/*"):
    replace_all_in_file(filename, plist)

In [ ]:
# Additional parameter entries
par.Cparameters("CHAR*", "NRPyEllipticET", "initial_lapse", "")
par.Cparameters("INT"  , "NRPyEllipticET", "lapse_exponent", -2)
pdict["bScale"        ] = "foci_position"
pdict["AMAX"          ] = "domain_size"
pdict["SINHWAA"       ] = "sinh_width"
pdict["initial_lapse" ] = "initial_lapse"
pdict["lapse_exponent"] = "lapse_exponent_n"

file1 = open(os.path.join(cfbbhdir, "set_Cparameters_from_parfile.h"), "w")
file2 = open(os.path.join(cfbbhdir, "set_Cparameters_from_parfile_paramstruct.h"), "w")
file1.write("// Set C parameters from user input\n")
file2.write("// Set C parameters from user input\n")
for key, val in pdict.items():
    string = f"params.{key} = {val}"+"\n"
    file1.write("griddata."+string)
    file2.write(string)
file1.close()
file2.close()

# Core driver function: `NRPyEllipticET.c`

In [ ]:
with open(os.path.join(srcdir, "NRPyEllipticET.c"), "w") as file:
    file.write(r"""// NRPyEllipticET header files
#include "NRPyEllipticET.h"

CCTK_REAL *NRPyEllipticET_uu = NULL;
CCTK_REAL *NRPyEllipticET_xx[3] = {NULL,NULL,NULL};

void NRPyEllipticET(CCTK_ARGUMENTS) {

  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // We only find the solution to the elliptic
  // problem on the first time this function is
  // called (typically on the coarsest Carpet
  // refinement level). The following if statement
  // takes care of that. Note, however, that the
  // solution will be obtained once per MPI process.
  if( NRPyEllipticET_uu == NULL ) {
    // Now solve the constraints of General Relativity
    if( CCTK_Equals(initial_data_type, "ConformallyFlatBBH") ) {
      NRPyEllipticET_conformally_flat_BBH_Hyperbolic_Relaxation();
    }
  }

  // Check memory has been allocated for NRPyEllipticET_uu. Error out otherwise.
  if( !NRPyEllipticET_uu || !NRPyEllipticET_xx[0] || !NRPyEllipticET_xx[1] || !NRPyEllipticET_xx[2] ) {
    CCTK_ERROR("Reached NRPyEllipticET_PunctureInitialData_Initialize_ADMBase() but memory for NRPyEllipticET_uu or NRPyEllipticET_xx has not been allocated.");
  }
  else {
    // Now interpolate the solution to the local ETK grid
    if( CCTK_Equals(initial_data_type, "ConformallyFlatBBH") ) {
      NRPyEllipticET_conformally_flat_BBH_Initialize_ADMBase(cctkGH,cctk_lsh,x,y,z,uuGF,
                                                             alp,betax,betay,betaz,
                                                             gxx,gxy,gxz,gyy,gyz,gzz,
                                                             kxx,kxy,kxz,kyy,kyz,kzz);
    }
  }
}
""")

# `NRPyEllipticET.h`

In [ ]:
with open(os.path.join(srcdir, "NRPyEllipticET.h"), "w") as file:
    file.write(r"""#ifndef NRPYELLIPTICET_H_
#define NRPYELLIPTICET_H_

// Standard C includes
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

// Einstein Toolkit interface
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

// This pointer will contain the solution for u
extern CCTK_REAL *NRPyEllipticET_uu;

// This pointer contains the numerical grid used by NRPyEllipticET
extern CCTK_REAL *NRPyEllipticET_xx[3];

// Function prototypes
// .-------------------------------------------------.
// | Conformally flat binary black hole initial data |
// .-------------------------------------------------.
void NRPyEllipticET_conformally_flat_BBH_Hyperbolic_Relaxation();
void NRPyEllipticET_conformally_flat_BBH_Initialize_ADMBase( const cGH *restrict cctkGH,
                                                             const int *restrict cctk_lsh,
                                                             const CCTK_REAL *restrict x,
                                                             const CCTK_REAL *restrict y,
                                                             const CCTK_REAL *restrict z,
                                                             CCTK_REAL *restrict uuGF,
                                                             CCTK_REAL *restrict alp,
                                                             CCTK_REAL *restrict betax,
                                                             CCTK_REAL *restrict betay,
                                                             CCTK_REAL *restrict betaz,
                                                             CCTK_REAL *restrict gxx,
                                                             CCTK_REAL *restrict gxy,
                                                             CCTK_REAL *restrict gxz,
                                                             CCTK_REAL *restrict gyy,
                                                             CCTK_REAL *restrict gyz,
                                                             CCTK_REAL *restrict gzz,
                                                             CCTK_REAL *restrict kxx,
                                                             CCTK_REAL *restrict kxy,
                                                             CCTK_REAL *restrict kxz,
                                                             CCTK_REAL *restrict kyy,
                                                             CCTK_REAL *restrict kyz,
                                                             CCTK_REAL *restrict kzz );

#endif // NRPYELLIPTICET_H_
""")

# `NRPyEllipticET_cleanup.c`

In [ ]:
with open(os.path.join(srcdir, "NRPyEllipticET_cleanup.c"), "w") as file:
    file.write(r"""#include "NRPyEllipticET.h"

void NRPyEllipticET_cleanup(CCTK_ARGUMENTS) {
  // Free memory allocated for NRPyEllipticET_uu and NRPyEllipticET_xx
  if( NRPyEllipticET_uu != NULL ) {
    free(NRPyEllipticET_uu); NRPyEllipticET_uu = NULL;
    for(int i=0;i<3;i++) {
      free(NRPyEllipticET_xx[i]); NRPyEllipticET_xx[i] = NULL;
    }
  }
  // Check everything is okay
  if( (NRPyEllipticET_uu    == NULL) &&
      (NRPyEllipticET_xx[0] == NULL) &&
      (NRPyEllipticET_xx[1] == NULL) &&
      (NRPyEllipticET_xx[2] == NULL) ) {
    CCTK_INFO("Successfully freed memory for NRPyEllipticET_uu and NRPyEllipticET_xx!");
  }
  else {
    CCTK_ERROR("Problem freeing memory for NRPyEllipticET_uu or NRPyEllipticET_xx!");
  }
  // All done!
  CCTK_INFO("All done!");
}
""")

# Base `make.code.defn`

In [ ]:
header = r"""# This file was generated by NRPy+ (http://nrpyplus.net)
# This and all other files used by the NRPyEllipticET thorn are
# fully documented in interactive Jupyter notebooks available at
#
# https://github.com/assumpcaothiago/NRPyElliptic
#
# Tutorial notebook documenting this file:
# Tutorial-ETK_thorn-NRPyEllipticET.ipynb
"""

with open(os.path.join(srcdir, "make.code.defn"), "w") as file:
    file.write(header+r"""
# Main make.code.defn file for thorn NRPyEllipticET

# Source files in this directory
SRCS = NRPyEllipticET.c NRPyEllipticET_cleanup.c

# Subdirectories containing source files
SUBDIRS = conformally_flat_BBH
""")

# CCL files

## `configuration.ccl`

In [ ]:
with open(os.path.join(thorndir, "configuration.ccl"), "w") as file:
    file.write(header+r"""
# Configuration for thorn NRPyEllipticET
PROVIDES NRPyEllipticET
{
}
""")

## `interface.ccl`

In [ ]:
with open(os.path.join(thorndir, "interface.ccl"), "w") as file:
    file.write(header+r"""
# Interface for thorn NRPyEllipticET
implements: NRPyEllipticET
inherits: ADMBase Grid

CCTK_REAL NRPyEllipticET_relaxation_vars TYPE=gf tags='prolongation="none"'
{
  uuGF
} "These are the variables NRPyEllipticET solves for. E.g., for conformally flat BBH, uuGF is the nonsingular part of the conformal factor."
""")

## `schedule.ccl`

In [ ]:
with open(os.path.join(thorndir, "schedule.ccl"), "w") as file:
    file.write(header+"""
# Schedule for thorn NRPyEllipticET
STORAGE: ADMBase::metric[metric_timelevels],ADMBase::curv[metric_timelevels],ADMBase::lapse[lapse_timelevels],ADMBase::shift[shift_timelevels]
STORAGE: NRPyEllipticET::NRPyEllipticET_relaxation_vars

schedule NRPyEllipticET IN ADMBase_InitialData
{
  LANG: C
  READS: Grid::x(Everywhere)
  READS: Grid::y(Everywhere)
  READS: Grid::z(Everywhere)
  WRITES: ADMBase::alp(Everywhere)
  WRITES: ADMBase::betax(Everywhere)
  WRITES: ADMBase::betay(Everywhere)
  WRITES: ADMBase::betaz(Everywhere)
  WRITES: ADMBase::kxx(Everywhere)
  WRITES: ADMBase::kxy(Everywhere)
  WRITES: ADMBase::kxz(Everywhere)
  WRITES: ADMBase::kyy(Everywhere)
  WRITES: ADMBase::kyz(Everywhere)
  WRITES: ADMBase::kzz(Everywhere)
  WRITES: ADMBase::gxx(Everywhere)
  WRITES: ADMBase::gxy(Everywhere)
  WRITES: ADMBase::gxz(Everywhere)
  WRITES: ADMBase::gyy(Everywhere)
  WRITES: ADMBase::gyz(Everywhere)
  WRITES: ADMBase::gzz(Everywhere)
} "Set up metric fields for binary black hole initial data"

schedule NRPyEllipticET_cleanup IN ADMBase_InitialData AFTER NRPyEllipticET
{
  LANG: C
  OPTIONS: GLOBAL-LATE
} "Deallocate memory for arrays used when computing initial data"
""")

## `param.ccl`

In [ ]:
with open(os.path.join(thorndir, "param.ccl"), "w") as file:
    file.write(header+"""
# Parameters for thorn NRPyEllipticET

#######################
# Extended parameters #
#######################
EXTENDS KEYWORD initial_data
{
  "NRPyEllipticET" :: "Initial data from NRPyEllipticET solution"
}

EXTENDS KEYWORD initial_shift
{
  "NRPyEllipticET" :: "Initial shift from NRPyEllipticET solution"
}

EXTENDS KEYWORD initial_dtlapse
{
  "NRPyEllipticET" :: "Initial dtlapse from NRPyEllipticET solution"
}

EXTENDS KEYWORD initial_dtshift
{
  "NRPyEllipticET" :: "Initial dtshift from NRPyEllipticET solution"
}

EXTENDS KEYWORD initial_lapse
{
  "NRPyEllipticET-psi^n"    :: "Based on the initial conformal factor"
  "NRPyEllipticET-averaged" :: "Averaged lapse for two puncture black holes"
}

######################
# General parameters #
######################
CCTK_STRING initial_data_type "Type of initial data generated by NRPyEllipticET"
{
  "ConformallyFlatBBH" :: "Conformally flat binary black hole initial data"
} "ConformallyFlatBBH"

BOOLEAN verbose "Whether or not to print information as the relaxation progresses"
{
} no

CCTK_INT info_output_freq "Print progress of relaxation time evolution every info_output_freq iterations"
{
  0:* :: "Any positive value. 0 disables it"
} 0

CCTK_REAL lapse_exponent_n "Exponent of lapse initial data alpha = psi^{n}"
{
  *:* :: "This parameter can have any value"
} -2

CCTK_STRING interpolator_name "Interpolator name"
{
  .+ :: "Can be anything"
} "Lagrange polynomial interpolation"

CCTK_INT interpolation_order "Interpolation order"
{
  0:* :: "Must be positive"
} 4

CCTK_STRING orbital_plane "Orbital plane"
{
  "xy" :: "Orbital plane is the xy-plane"
  "yx" :: "Equivalent to xy"
  "xz" :: "Orbital plane is the xz-plane"
  "zx" :: "Equivalent to xz"
} "xy"

CCTK_REAL position_shift[3] "Shift punctures on the axis while keeping their distance fixed"
{
  *:* :: "Can be anything"
} 0

####################################
# Hyperbolic relaxation parameters #
####################################
CCTK_REAL eta_damping "Wave equation damping parameter"
{
  0:* :: "Must be positive"
  -1  :: "Compute optimum eta_damping (requires domain_size=1e6; sinh_width=0.07; foci_position=5)"
} -1

CCTK_REAL number_of_LCT "Number of light-crossing times"
{
  0:* :: "Must be positive"
} 5.5e-6

CCTK_REAL log_target_residual "Stopping criterion: log10(l2norm(residual))"
{
  *:0 :: "Should be negative, since we typically want small residuals"
} -16.0

CCTK_REAL NRPy_epsilon "Small number to avoid singularities at the punctures"
{
  *:* :: "Can be anything"
} 1e-8

CCTK_REAL CFL_FACTOR "CFL factor of the relaxation time evolution."
{
  0:1 :: "Between zero and one. For higher resolutions this will have to be lowered."
} 0.7

CCTK_INT max_iterations "Maximum number of time steps in the relaxation time evolution."
{
  0:* :: "Must be positive"
} 10000

CCTK_REAL residual_integration_radius "The L2-norm of the residual is integrated in a sphere with this radius."
{
  0:* :: "Must be positive; if set to >=domain_size then will integrate over the entire NRPy+-generated grid."
} 100.0

############################################
# Conformally flat puncture BBH parameters #
############################################
CCTK_REAL conformally_flat_BBH_puncture_0_bare_mass "Bare mass of first puncture"
{
  0:* :: "Must be positive"
} 0.5

CCTK_REAL conformally_flat_BBH_puncture_1_bare_mass "Bare mass of second puncture"
{
  0:* :: "Must be positive"
} 0.5

CCTK_REAL conformally_flat_BBH_puncture_0_pos[3] "Position of first puncture"
{
  *:* :: "This parameter can have any value"
} 0

CCTK_REAL conformally_flat_BBH_puncture_1_pos[3] "Position of second puncture"
{
  *:* :: "This parameter can have any value"
} 0

CCTK_REAL conformally_flat_BBH_puncture_0_P[3] "Linear momentum of first puncture"
{
  *:* :: "This parameter can have any value"
} 0

CCTK_REAL conformally_flat_BBH_puncture_1_P[3] "Linear momentum of second puncture"
{
  *:* :: "This parameter can have any value"
} 0

CCTK_REAL conformally_flat_BBH_puncture_0_S[3] "Angular momentum of first puncture"
{
  *:* :: "This parameter can have any value"
} 0

CCTK_REAL conformally_flat_BBH_puncture_1_S[3] "Angular momentum of second puncture"
{
  *:* :: "This parameter can have any value"
} 0
""")

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish-NRPyEllipticET")